In [13]:
pkgs <- c("ISLR", "gclus", "assertthat", "ggplot2")
for (p in pkgs) {
    if (!requireNamespace(p, quietly = TRUE)) install.packages(p, repos = "https://cran.rstudio.com")
    library(p, character.only = TRUE)
}

options(repr.plot.width=20, repr.plot.height=5)

options(scipen=999)
set.seed(42)

In [14]:
results <- read.csv("../data/results.csv")
head(results)
summary(results)

,alpha_ent,k,n_topics,n_neighbours,alpha_word,score_deezer,score_itunes
,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,0.3,20,10,5,0.2,48.6781,51.6682
2,0.3,20,10,5,0.3,48.6781,51.6682
3,0.3,20,10,5,0.4,49.6023,50.9951
4,0.3,20,10,5,0.5,49.8568,48.9428
5,0.3,20,10,10,0.2,52.9038,51.3696
6,0.3,20,10,10,0.3,52.9038,51.3696


   alpha_ent         k            n_topics      n_neighbours     alpha_word   
 Min.   :0.3   Min.   : 20.0   Min.   : 10.0   Min.   :  5.0   Min.   :0.200  
 1st Qu.:0.3   1st Qu.: 20.0   1st Qu.: 10.0   1st Qu.:  5.0   1st Qu.:0.275  
 Median :0.3   Median : 50.0   Median : 20.0   Median : 10.0   Median :0.350  
 Mean   :0.3   Mean   : 62.4   Mean   : 43.6   Mean   :128.7   Mean   :0.350  
 3rd Qu.:0.3   3rd Qu.:100.0   3rd Qu.: 50.0   3rd Qu.: 20.0   3rd Qu.:0.425  
 Max.   :0.3   Max.   :200.0   Max.   :100.0   Max.   :500.0   Max.   :0.500  
  score_deezer    score_itunes  
 Min.   :47.91   Min.   :47.63  
 1st Qu.:50.46   1st Qu.:50.09  
 Median :52.09   Median :51.43  
 Mean   :52.14   Mean   :51.27  
 3rd Qu.:53.48   3rd Qu.:52.58  
 Max.   :56.58   Max.   :54.40  

In [ ]:
# experiments from paper:
# - alpha_ent: 0.30 0.40
# - k: 20 50 100 200 (default=500 is not used in paper)
# - alpha_word: 0.2 0.3 0.4 0.5

# common values for undefined parameters:
# - n_topics: 10 20 50 100
# - n_neighbours: 5 10 20 500